In [10]:
import torch
import torch.optim as optim
import os
import torch.nn as nn
import sys

sys.path.append('../')
from models import LatentMLP, VAE
from utils import BrainGraphDataset, project_root

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
root = project_root()

# instantiate the VGAE model
hidden_dim = 128
latent_dim = 64
input_dim = 4950
output_dim = 1
lr = 0.001
batch_size = 8

vae = VAE(input_dim, [128] * 2, latent_dim)

# load the trained VGAE weights
vae.load_state_dict(torch.load(os.path.join(root, 'vae_weights/vae_dropout_psilo_schaefer_before_0.pt'), map_location=device))

# define the optimizer and the loss function

criterion = nn.L1Loss(reduction='sum')

dataroot = 'fc_matrices/psilo_schaefer_before'


# Convert the model to the device
vae.to(device)

annotations = 'annotations.csv'
psilo_dataset = BrainGraphDataset(img_dir=os.path.join(root, dataroot),
                            annotations_file=os.path.join(root, annotations),
                            transform=None, extra_data=None, setting='upper_triangular_baseline_lz')

# Define the train, validation, and test ratios
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Get the number of samples in the dataset
num_samples = len(psilo_dataset)

# Calculate the number of samples for each set
train_size = int(train_ratio * num_samples)
val_size = int(val_ratio * num_samples)
test_size = num_samples - train_size - val_size

torch.manual_seed(0)
# Split the dataset into train, validation, and test sets
train_set, val_set, test_set = torch.utils.data.random_split(psilo_dataset, [train_size, val_size, test_size])

# Define the dataloaders for each set

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=val_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_size, shuffle=False)

num_epochs = 300


import json

# Dictionary to store training and validation curves
curves = {}


# instantiate the LatentMLP model
mlp = LatentMLP(latent_dim, hidden_dim, output_dim, dropout=0, extra_dim=100)
optimizer = optim.Adam(mlp.parameters(), lr=lr)
# Convert the MLP to the device
mlp.to(device)

best_val_loss = float('inf')
best_mlp_state = None

# Lists to store training and validation losses
train_losses = []
val_losses = []

print('Started Training')
# train the MLP on the new dataset
for epoch in range(num_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # get the inputs
        (graphs, base_bdis, lz), labels = data

        graphs = graphs.to(device)
        base_bdis = base_bdis.to(device)
        lz = lz.to(device).view(-1, 100)

        base_bdis = torch.cat([base_bdis, lz], dim=1)

        labels = labels.to(device).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # get the latent embeddings from the VGAE
        _, _, _, zs = vae(graphs.view(-1, input_dim))

        # pass the latent embeddings through the MLP

        outputs = mlp(zs, base_bdis)

        # calculate the loss and backpropagate
        loss = mlp.loss(outputs, labels.view(outputs.shape))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    # Validation check
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            (graphs, base_bdis, lz), labels = data

            graphs = graphs.to(device)
            base_bdis = base_bdis.to(device)
            lz = lz.to(device).view(-1, 100)
            base_bdis = torch.cat([base_bdis, lz], dim=1)

            labels = labels.to(device).float()

            # zero the parameter gradients
            optimizer.zero_grad()

            # get the latent embeddings from the VGAE
            _, _, _, zs = vae(graphs.view(-1, input_dim))

            # pass the latent embeddings through the MLP
            outputs = mlp(zs, base_bdis)
            val_loss += criterion(outputs, labels.view(outputs.shape)).item()
    val_loss /= len(test_set)

    # Save the best model so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_mlp_state = mlp.state_dict().copy()

    # Print statistics and perform testing every 5 epochs
    if (epoch + 1) % 1 == 0:
        print('[%d, %5d] loss: %.3f, val_loss: %.3f' %
              (epoch + 1, i + 1, running_loss / (len(train_set)), val_loss))
        running_loss = 0.0

    train_losses.append(running_loss / (len(train_set)))
    val_losses.append(val_loss)
                            
#     # Add the dropout and its curves to the dictionary
#     curves[str(dropout)] = {
#         'train_loss': train_losses,
#         'val_loss': val_losses
#     }
    
#     torch.save(best_mlp_state, f'mlp_weights/dropout_{dropout}')

# # Save the curves dictionary to a JSON file
# with open('mlp_dropout_tests.json', 'w') as f:
#     json.dump(curves, f)

print('Finished Training')

Started Training
[1,     4] loss: 11.966, val_loss: 9.092
[2,     4] loss: 10.542, val_loss: 7.527
[3,     4] loss: 8.796, val_loss: 6.189
[4,     4] loss: 7.558, val_loss: 6.341
[5,     4] loss: 8.068, val_loss: 6.605
[6,     4] loss: 7.974, val_loss: 5.883
[7,     4] loss: 7.055, val_loss: 5.761
[8,     4] loss: 7.160, val_loss: 5.872
[9,     4] loss: 7.242, val_loss: 5.854
[10,     4] loss: 7.168, val_loss: 5.716
[11,     4] loss: 6.997, val_loss: 5.779
[12,     4] loss: 6.985, val_loss: 5.750
[13,     4] loss: 6.961, val_loss: 5.731
[14,     4] loss: 6.932, val_loss: 5.762
[15,     4] loss: 6.916, val_loss: 5.768
[16,     4] loss: 6.903, val_loss: 5.811
[17,     4] loss: 7.018, val_loss: 5.846
[18,     4] loss: 6.859, val_loss: 5.769
[19,     4] loss: 6.957, val_loss: 5.798
[20,     4] loss: 6.975, val_loss: 5.875
[21,     4] loss: 6.870, val_loss: 5.869
[22,     4] loss: 6.829, val_loss: 5.835
[23,     4] loss: 6.846, val_loss: 5.886
[24,     4] loss: 6.827, val_loss: 5.836
[25,  

Traceback (most recent call last):
  File "/rds/general/user/ljn19/home/anaconda3/envs/fyp/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/tmp/pbs.7724891.pbs/ipykernel_2246010/670334633.py", line 92, in <module>
    for i, data in enumerate(train_loader, 0):
  File "/rds/general/user/ljn19/home/anaconda3/envs/fyp/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/rds/general/user/ljn19/home/anaconda3/envs/fyp/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 678, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/rds/general/user/ljn19/home/anaconda3/envs/fyp/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/rds/general/user/ljn19/home/anaconda3/

In [ ]:
import json
import matplotlib.pyplot as plt

# Load the curves from the JSON file
with open('mlp_dropout_tests.json', 'r') as f:
    curves = json.load(f)

# Plot the curves for each dropout value

epochs = range(1, len(train_losses) + 1)

# Plot training curve
plt.figure()

for dropout, curve_data in curves.items():
    train_losses = curve_data['train_loss']
    plt.plot(epochs, train_losses, label=f'Training Loss (Dropout {dropout})')

plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title(f'Training Curve')
plt.legend()

# Plot validation curve
plt.figure()

for dropout, curve_data in curves.items():
    val_losses = curve_data['val_loss']
    plt.plot(epochs, val_losses, label=f'Validation Loss (Dropout {dropout})')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.title(f'Validation Curve')
plt.legend()

# Show the plots
plt.show()


In [ ]:
# Load the best model and perform final testing

import matplotlib.pyplot as plt
import numpy as np

# Evaluate the model on the test set and plot the results

train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_size, shuffle=True)

names = ['Train', 'Test']
datasets = [train_set, test_set]
loaders = [train_loader, test_loader]

for dropout in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
    
    mlp.load_state_dict(torch.load(f'mlp_weights/dropout_{dropout}.pt', map_location=device))
    test_loss = 0.0

    for i in range(2):
        with torch.no_grad():
            for data in loaders[i]:
                (graphs, base_bdis), labels = data

                graphs = graphs.to(device)
                base_bdis = base_bdis.to(device)

                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # get the latent embeddings from the VGAE
                _, _, _, zs = vae(graphs.view(-1, input_dim))

                # pass the latent embeddings through the MLP

                outputs = mlp(zs, base_bdis)
                criterion = nn.L1Loss(reduction='sum')
                test_loss += criterion(outputs, labels.view(outputs.shape)).item()

                # Convert back to CPU and numpy arrays for plotting
                labels = labels.cpu().numpy().reshape(-1)
                predictions = outputs.cpu().numpy().reshape(-1)

                # Plot the results
                plt.scatter(labels, predictions)
                plt.xlabel('True values')
                plt.ylabel('Predicted values')
                plt.title(f'Predicted vs true BDI, dropout {dropout} ({names[i]})')
                plt.xlim([0,50])
                plt.ylim([0,50])
                plt.show()

        # Print the test loss
        test_loss /= len(datasets[i])
        print(f'{names[i]} loss: %.3f' % test_loss)
        correlation_matrix = np.corrcoef(labels, predictions)
        pearson_coefficient = correlation_matrix[0, 1]
        print(f'Pearson coefficient: {pearson_coefficient}')


In [ ]:
names = ['fine_tune', 'mixed_class']
infiles = ['fine_tune_best.pt', 'mixed_class_0.05_0.01_best.pt']
outfiles = ['fine_tune_mlp_dropout_0.1', 'mixed_class_mlp_dropout_0.1']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# instantiate the VGAE model
hidden_dim = 256
latent_dim = 64
input_dim = 100 * 100
output_dim = 1
lr = 0.001
batch_size = 8

# for j in range(2):
#     vae = VAEReg(input_dim, hidden_dim, latent_dim, dropout=0.1)
#     # load the trained VGAE weights
#     vae.load_state_dict(torch.load(f'vgae_weights/{infiles[j]}', map_location=device))

#     # Convert the model to the device
#     vae.to(device)

#     # freeze the weights of the VAE
#     for param in vae.parameters():
#         param.requires_grad = False

#     # define the optimizer and the loss function

#     criterion = nn.L1Loss(reduction='sum')

#     dataroot = 'fc_matrices/psilo_ica_100_after'
#     cwd = os.getcwd() + '/'

#     num_epochs = 300

#     import json

#     # Dictionary to store training and validation curves
#     curves = {}

#     # instantiate the LatentMLP model
#     mlp = LatentMLP(latent_dim, hidden_dim, output_dim, dropout=0.1)
#     optimizer = optim.Adam(mlp.parameters(), lr=lr)
#     # Convert the MLP to the device
#     mlp.to(device)

#     best_val_loss = float('inf')
#     best_mlp_state = None

#     # Lists to store training and validation losses
#     train_losses = []
#     val_losses = []

#     num_epochs = 1000
#     # train the MLP on the new dataset
#     for epoch in range(num_epochs):
#         running_loss = 0.0

#         for i, data in enumerate(train_loader, 0):
#             # get the inputs
#             (graphs, base_bdis), labels = data

#             graphs = graphs.to(device)
#             base_bdis = base_bdis.to(device)

#             labels = labels.to(device).float()

#             # zero the parameter gradients
#             optimizer.zero_grad()

#             # get the latent embeddings from the VGAE
#             _, _, _, zs = vae(graphs.view(-1, input_dim))

#             # pass the latent embeddings through the MLP

#             outputs = mlp(zs, base_bdis)

#             # calculate the loss and backpropagate
#             loss = mlp.loss(outputs, labels.view(outputs.shape))
#             loss.backward()
#             optimizer.step()

#             # print statistics
#             running_loss += loss.item()

#         # Validation check
#         val_loss = 0.0
#         with torch.no_grad():
#             for data in val_loader:
#                 (graphs, base_bdis), labels = data

#                 graphs = graphs.to(device)
#                 base_bdis = base_bdis.to(device)

#                 labels = labels.to(device).float()

#                 # zero the parameter gradients
#                 optimizer.zero_grad()

#                 # get the latent embeddings from the VGAE
#                 _, _, _, zs = vae(graphs.view(-1, input_dim))

#                 # pass the latent embeddings through the MLP
#                 outputs = mlp(zs, base_bdis)
#                 val_loss += criterion(outputs, labels.view(outputs.shape)).item()
#         val_loss /= len(val_set)

#         train_losses.append(running_loss / len(train_set))
#         val_losses.append(val_loss)
        
#         # Save the best model so far
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             best_mlp_state = mlp.state_dict().copy()

#         # Print statistics and perform testing every 5 epochs
#         if epoch % 10 == 9:
#             print('[%d, %5d] loss: %.3f, val_loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / (len(train_set)), val_loss))
#             running_loss = 0.0
            
#     curves[names[j]] = {'val_losses': val_losses, 'train_losses': train_losses}
#     torch.save(best_mlp_state, f'mlp_weights/{outfiles[j]}.pt')
    
#     with open('mlp_hyper_curves.json', 'w') as f:
#         json.dump(curves, f)
# print('Finished Training')


In [ ]:
import json
import matplotlib.pyplot as plt

# load in the loss curves from file
with open("mlp_hyper_curves.json", "r") as f:
    loss_curves = json.load(f)

val_losses = loss_curves['mixed_class']['val_losses']
# plot the validation loss curves for each number of GMM components
plt.figure(figsize=(8, 6))

epochs = range(1, len(val_losses) + 1)
plt.plot(epochs, val_losses, label="Val")

# add labels and legend
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.title("Validation Loss Curves")
plt.legend()
plt.ylim((2.5, 15))

# show the plot
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt

# load in the loss curves from file
with open("mlp_hyper_curves.json", "r") as f:
    loss_curves = json.load(f)

train_losses = loss_curves['mixed_class']['train_losses']
# plot the validation loss curves for each number of GMM components
plt.figure(figsize=(8, 6))

epochs = range(1, len(train_losses) + 1)
plt.plot(epochs, train_losses, label="Train")

# add labels and legend
plt.xlabel("Epoch")
plt.ylabel("Train Loss")
plt.title("Train Loss Curves")
plt.legend()
plt.ylim((0, 15))

# show the plot
plt.show()

In [ ]:
#train on the fine tune vae
names = ['Train', 'Test']

for mlp_path in ['fine_tune_mlp_dropout_0.1.pt', 'mixed_class_mlp_dropout_0.1.pt']:
    mlp.load_state_dict(torch.load(f'mlp_weights/{mlp_path}', map_location=device))
    test_loss = 0.0

    for i in range(2):
        with torch.no_grad():
            for data in loaders[i]:
                (graphs, base_bdis), labels = data

                graphs = graphs.to(device)
                base_bdis = base_bdis.to(device)

                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # get the latent embeddings from the VGAE
                _, _, _, zs = vae(graphs.view(-1, input_dim))

                # pass the latent embeddings through the MLP

                outputs = mlp(zs, base_bdis)
                criterion = nn.L1Loss(reduction='sum')
                test_loss += criterion(outputs, labels.view(outputs.shape)).item()

                # Convert back to CPU and numpy arrays for plotting
                labels = labels.cpu().numpy().reshape(-1)
                predictions = outputs.cpu().numpy().reshape(-1)

                # Plot the results
                plt.scatter(labels, predictions)
                plt.xlabel('True values')
                plt.ylabel('Predicted values')
                plt.title(f'Predicted vs true BDI ({names[i]})')
                plt.xlim([0,50])
                plt.ylim([0,50])
                plt.show()

        # Print the test loss
        test_loss /= len(datasets[i])
        print(f'{names[i]} loss: %.3f' % test_loss)
        correlation_matrix = np.corrcoef(labels, predictions)
        pearson_coefficient = correlation_matrix[0, 1]
        print(f'Pearson coefficient: {pearson_coefficient}')